In [185]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
gender_submission = pd.read_csv('gender_submission.csv')
gender_submission['Survived'] = 0

train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [186]:
# 실제 데이터셋 전처리

train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
train_df['survival expect'] = 0


bins = [0, 18, 35, 60, 100]
labels = ['미성년자', '청년', '중년', '노년']
train_df['Age'] = pd.cut(train_df['Age'], bins=bins, labels=labels)


q1 = train_df['Fare'].quantile(0.25)
q2 = train_df['Fare'].quantile(0.50)
q3 = train_df['Fare'].quantile(0.75)
bins = [0, q1, q2, q3, train_df['Fare'].max()]
labels = ['Low', 'Medium', 'High', 'Very High']
train_df['Fare'] = pd.cut(train_df['Fare'], bins=bins, labels=labels, include_lowest=True)

train_df

C:\Users\Seunghwan\AppData\Local\Temp\ipykernel_23788\180235696.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,survival expect
0,1,0,3,"Braund, Mr. Owen Harris",male,청년,1,0,A/5 21171,Low,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,중년,1,0,PC 17599,Very High,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,청년,0,0,STON/O2. 3101282,Medium,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,청년,1,0,113803,Very High,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,청년,0,0,373450,Medium,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,청년,0,0,211536,Medium,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,청년,0,0,112053,High,B42,S,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,청년,1,2,W./C. 6607,High,NaN,S,0
889,890,1,1,"Behr, Mr. Karl Howell",male,청년,0,0,111369,High,C148,C,0


In [187]:
# 그룹별 생존율

pclass_sur =  train_df.groupby('Pclass')['Survived'].value_counts()
pclass_sur_rate = train_df.groupby('Pclass')['Survived'].mean() * 100
pclass_sur_rate_mean = pclass_sur_rate.mean()


sex_sur =  train_df.groupby('Sex')['Survived'].value_counts()
sex_sur_rate = train_df.groupby('Sex')['Survived'].mean() * 100
sex_sur_rate_mean = sex_sur_rate.mean()


age_sur =  train_df.groupby('Age')['Survived'].value_counts()
age_sur_rate = train_df.groupby('Age')['Survived'].mean() * 100
age_sur_rate_mean = age_sur_rate.mean()


sibsp_sur =  train_df.groupby('SibSp')['Survived'].value_counts()
sibsp_sur_rate = train_df.groupby('SibSp')['Survived'].mean() * 100
sibsp_sur_rate_mean = sibsp_sur_rate.mean()


parch_sur =  train_df.groupby('Parch')['Survived'].value_counts()
parch_sur_rate = train_df.groupby('Parch')['Survived'].mean() * 100
parch_sur_rate_mean = parch_sur_rate.mean()


fare_sur =  train_df.groupby('Fare')['Survived'].value_counts()
fare_sur_rate = train_df.groupby('Fare')['Survived'].mean() * 100
fare_sur_rate_mean = fare_sur_rate.mean()


embarked_sur =  train_df.groupby('Embarked')['Survived'].value_counts()
embarked_sur_rate = train_df.groupby('Embarked')['Survived'].mean() * 100
embarked_sur_rate_mean = embarked_sur_rate.mean()

C:\Users\Seunghwan\AppData\Local\Temp\ipykernel_23788\3274324842.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_sur =  train_df.groupby('Age')['Survived'].value_counts()
C:\Users\Seunghwan\AppData\Local\Temp\ipykernel_23788\3274324842.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_sur_rate = train_df.groupby('Age')['Survived'].mean() * 100
C:\Users\Seunghwan\AppData\Local\Temp\ipykernel_23788\3274324842.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True 

In [188]:
# 평균 초과 집단 추출

def compare_to_mean(rate, mean):
    higher = rate[rate > mean]
    return higher


pclass_higher = compare_to_mean(pclass_sur_rate, pclass_sur_rate_mean)
sex_higher = compare_to_mean(sex_sur_rate, sex_sur_rate_mean)
age_higher = compare_to_mean(age_sur_rate, age_sur_rate_mean)
sibsp_higher = compare_to_mean(sibsp_sur_rate, sibsp_sur_rate_mean)
parch_higher = compare_to_mean(parch_sur_rate, parch_sur_rate_mean)
fare_higher = compare_to_mean(fare_sur_rate, fare_sur_rate_mean)
embarked_higher = compare_to_mean(embarked_sur_rate, embarked_sur_rate_mean)


# train_df 에서 초과하는 집단에 속한 행에 +1

train_df.loc[train_df['Pclass'].isin(pclass_higher.index), 'survival expect'] += 1
train_df.loc[train_df['Sex'].isin(sex_higher.index), 'survival expect'] += 1
train_df.loc[train_df['Age'].isin(age_higher.index), 'survival expect'] += 1
train_df.loc[train_df['SibSp'].isin(sibsp_higher.index), 'survival expect'] += 1
train_df.loc[train_df['Parch'].isin(parch_higher.index), 'survival expect'] += 1
train_df.loc[train_df['Fare'].isin(fare_higher.index), 'survival expect'] += 1
train_df.loc[train_df['Embarked'].isin(embarked_higher.index), 'survival expect'] += 1


# 최대 7점 중 5점 이상 생존으로 결정

def decide_survival(x):
    if x >= 5:
        return 1
    else:
        return 0


train_df['survival expect'] = train_df['survival expect'].apply(decide_survival)

train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,survival expect
0,1,0,3,"Braund, Mr. Owen Harris",male,청년,1,0,A/5 21171,Low,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,중년,1,0,PC 17599,Very High,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,청년,0,0,STON/O2. 3101282,Medium,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,청년,1,0,113803,Very High,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,청년,0,0,373450,Medium,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,청년,0,0,211536,Medium,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,청년,0,0,112053,High,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,청년,1,2,W./C. 6607,High,NaN,S,0
889,890,1,1,"Behr, Mr. Karl Howell",male,청년,0,0,111369,High,C148,C,1


In [189]:
# 실제 생존 여부와 예측한 생존 여부 비교

train_df[train_df.Survived == train_df['survival expect']]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,survival expect
0,1,0,3,"Braund, Mr. Owen Harris",male,청년,1,0,A/5 21171,Low,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,중년,1,0,PC 17599,Very High,C85,C,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,청년,1,0,113803,Very High,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,청년,0,0,373450,Medium,NaN,S,0
5,6,0,3,"Moran, Mr. James",male,청년,0,0,330877,Medium,NaN,Q,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,청년,0,0,211536,Medium,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,청년,0,0,112053,High,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,청년,1,2,W./C. 6607,High,NaN,S,0
889,890,1,1,"Behr, Mr. Karl Howell",male,청년,0,0,111369,High,C148,C,1


In [190]:
# 테스트 데이터셋에 적용 시작

test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [191]:
# 테스트 데이터셋 전처리

test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)
test_df['Survived'] = 0


bins = [0, 18, 35, 60, 100]
labels = ['미성년자', '청년', '중년', '노년']
test_df['Age'] = pd.cut(test_df['Age'], bins=bins, labels=labels)


q1 = test_df['Fare'].quantile(0.25)
q2 = test_df['Fare'].quantile(0.50)
q3 = test_df['Fare'].quantile(0.75)
bins = [0, q1, q2, q3, test_df['Fare'].max()]
labels = ['Low', 'Medium', 'High', 'Very High']
test_df['Fare'] = pd.cut(test_df['Fare'], bins=bins, labels=labels, include_lowest=True)

C:\Users\Seunghwan\AppData\Local\Temp\ipykernel_23788\2183595552.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)


In [192]:
test_df.loc[test_df['Pclass'].isin(pclass_higher.index), 'Survived'] += 1
test_df.loc[test_df['Sex'].isin(sex_higher.index), 'Survived'] += 1
test_df.loc[test_df['Age'].isin(age_higher.index), 'Survived'] += 1
test_df.loc[test_df['SibSp'].isin(sibsp_higher.index), 'Survived'] += 1
test_df.loc[test_df['Parch'].isin(parch_higher.index), 'Survived'] += 1
test_df.loc[test_df['Fare'].isin(fare_higher.index), 'Survived'] += 1
test_df.loc[test_df['Embarked'].isin(embarked_higher.index), 'Survived'] += 1


def decide_survival(x):
    if x >= 5:
        return 1
    else:
        return 0


test_df['Survived'] = test_df['Survived'].apply(decide_survival)

test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,청년,0,0,330911,Low,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,중년,1,0,363272,Low,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,노년,0,0,240276,Medium,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,청년,0,0,315154,Medium,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,청년,1,1,3101298,Medium,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,청년,0,0,A.5. 3236,Medium,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,중년,0,0,PC 17758,Very High,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,중년,0,0,SOTON/O.Q. 3101262,Low,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,청년,0,0,359309,Medium,NaN,S,0


In [193]:
# 답안지 작성

gender_submission['Survived'] = test_df['Survived']
gender_submission.to_csv('my_submission.csv', index=False)

gender_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
